In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob as gb
import cv2

In [2]:
data=pd.read_csv('../input/dfl-bundesliga-data-shootout/train.csv')

In [3]:
data.head()

,video_id,time,event,event_attributes
0,1606b0e6_0,200.265822,start,NaN
1,1606b0e6_0,201.150000,challenge,['ball_action_forced']
2,1606b0e6_0,202.765822,end,NaN
3,1606b0e6_0,210.124111,start,NaN
4,1606b0e6_0,210.870000,challenge,['opponent_dispossessed']


In [4]:
data.drop(['event_attributes'],inplace=True,axis=1)

In [5]:
data.head()

,video_id,time,event
0,1606b0e6_0,200.265822,start
1,1606b0e6_0,201.150000,challenge
2,1606b0e6_0,202.765822,end
3,1606b0e6_0,210.124111,start
4,1606b0e6_0,210.870000,challenge


In [6]:
data_challenge=data[data['event']=='challenge']

In [7]:
data_challenge.head()

,video_id,time,event
1,1606b0e6_0,201.15,challenge
4,1606b0e6_0,210.87,challenge
59,1606b0e6_0,300.99,challenge
62,1606b0e6_0,307.27,challenge
79,1606b0e6_0,395.55,challenge


In [8]:
data_challenge.shape

(624, 3)

In [9]:
data_play=data[data['event']=='play']

In [10]:
data_play.head()

,video_id,time,event
10,1606b0e6_0,224.43,play
13,1606b0e6_0,229.39,play
16,1606b0e6_0,236.71,play
17,1606b0e6_0,239.35,play
20,1606b0e6_0,242.39,play


In [11]:
data_play.shape

(3586, 3)

In [12]:
data_throwin=data[data['event']=='throwin']

In [13]:
data_throwin.head()

,video_id,time,event
7,1606b0e6_0,219.23,throwin
101,1606b0e6_0,427.67,throwin
119,1606b0e6_0,474.39,throwin
162,1606b0e6_0,621.11,throwin
265,1606b0e6_0,875.39,throwin


In [14]:
data_throwin.shape

(172, 3)

In [15]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 40.7 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110743 sha256=0ed73cdb015ccea49a8ad430f1f27038d780fea401648d24fee651b60985b4da
  Stored in directory: /root/.cache/pip/wheels/56/dc/2b/9cd600d483c04af3353d66623056fc03faed76b7518faae4df
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2022.5.0 requires fsspec==2022.5.0, but you have fsspec 2022.8.2 which is incompatible.


In [16]:
os.makedirs('dataset')
os.makedirs('dataset/play')
os.makedirs('dataset/throwin')
os.makedirs('dataset/challenge')

In [17]:
#import shutil
#shutil.rmtree("./dataset")

In [18]:
play_path='./dataset/play'
throwin_path='./dataset/throwin'
challenge_path='./dataset/challenge'

In [19]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [20]:
data_throwin.shape[0]

172

In [21]:
import time
def split_video_train(event_data,video_path,path_save_images,time_before_event,time_after_event,path_cutvideo,skip_frame):
    count=0
    skip_frames=0
    start_time=time.time()
    print('start time : ' , start_time)
    for i in range(event_data.shape[0]):
        index_video=event_data.iloc[i,0]
        index_time=event_data.iloc[i,1]
        ffmpeg_extract_subclip(f"{video_path}/{index_video}.mp4" , index_time - time_before_event, index_time + time_after_event , targetname="cutvideo.mp4")
        cap=cv2.VideoCapture(f'{path_cutvideo}/cutvideo.mp4')
        print('strart')
        while cap.isOpened() :
            ret,image=cap.read()
            if ret== False :
                break 
            if skip_frames % skip_frame == 0 :
                cv2.imwrite(f'{path_save_images}/{index_video}_{count}.jpg',image)
                count  = count + 1 
            skip_frames = skip_frames + 1
        cap.release()
        print('end')
    print('number of frames : ',count)
    print('end time : ', time.time() - start_time)

In [22]:
split_video_train(event_data = data_throwin
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = throwin_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 3
                 )

start time :  1663511198.2171566
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy

In [23]:
split_video_train(event_data = data_challenge
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = challenge_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 3
                 )

start time :  1663511320.831539
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy 

In [24]:
split_video_train(event_data = data_play
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = play_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 3
                 )

start time :  1663511774.5357625
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
strart
end
Moviepy

In [1]:
print()